In [3]:
import openmeteo_requests

import requests_cache
import pandas as pd

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from retry_requests import retry
import pickleshare

In [55]:
def call_open_meteo_averages(selected_station):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": selected_station["lat"],
        "longitude": selected_station["lon"],
        "start_date": "1999-12-01",
        "end_date": "1999-12-31",
        "hourly": "wind_speed_10m"
    }
    responses = openmeteo.weather_api(url, params=params)
    
    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    
    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_wind_speed_10m = hourly.Variables(0).ValuesAsNumpy()
    
    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    
    hourly_dataframe = pd.DataFrame(data = hourly_data)
    # Group hourly data by day and calculate the daily average wind speed
    daily_averages = hourly_dataframe.resample('D', on='date').mean()
    # Extract only the date from the index
    daily_averages.index = daily_averages.index.date
    
    # Print the daily averages
    return daily_averages


In [39]:
def call_open_meteo_min(selected_station):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
    retry_strategy = Retry(total=5, backoff_factor=0.2)
    adapter = HTTPAdapter(max_retries=retry_strategy)
    cache_session.mount('http://', adapter)
    cache_session.mount('https://', adapter)
    openmeteo = openmeteo_requests.Client(session=cache_session)

    # Initialize an empty list to store daily data
    daily_data = []

    # Define the start and end dates for the loop
    start_date = pd.Timestamp("1999-12-01")
    end_date = pd.Timestamp("1999-12-31")

    # Iterate through each day of December
    while start_date <= end_date:
        # Define the parameters for the API call
        params = {
            "latitude": selected_station["lat"],
            "longitude": selected_station["lon"],
            "start_date": start_date.strftime("%Y-%m-%d"),
            "end_date": start_date.strftime("%Y-%m-%d"),  # Use the same start and end date for daily data
            "daily": "wind_speed_10m_min",
            "timezone": "Europe/Berlin",
        }

        # Make the API call
        responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)

        for response in responses:
            # Process daily data. The order of variables needs to be the same as requested.
            daily = response.Daily()
            daily_wind_speed_10m_min = daily.Variables(0).ValuesAsNumpy()

            # Calculate the average wind speed for the day
            min_wind_speed = daily_wind_speed_10m_min.min()

            # Append the average wind speed for the day to daily_data
            daily_data.append({
                "date": start_date.strftime('%Y-%m-%d'),  # Format date to include year, month, and day
                "wind_speed_10m_max": min_wind_speed,  # Use the average wind speed
            })

        # Move to the next day
        start_date += pd.Timedelta(days=1)

    # Create a DataFrame from daily data
    daily_dataframe = pd.DataFrame(daily_data)
    
    # Calculate daily averages
    daily_min = daily_dataframe.groupby('date').min()

    return daily_min


In [48]:
def call_open_meteo_max(selected_station):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
    retry_strategy = Retry(total=5, backoff_factor=0.2)
    adapter = HTTPAdapter(max_retries=retry_strategy)
    cache_session.mount('http://', adapter)
    cache_session.mount('https://', adapter)
    openmeteo = openmeteo_requests.Client(session=cache_session)

    # Initialize an empty list to store daily data
    daily_data = []

    # Define the start and end dates for the loop
    start_date = pd.Timestamp("1999-12-01")
    end_date = pd.Timestamp("1999-12-31")

    # Iterate through each day of December
    while start_date <= end_date:
        # Define the parameters for the API call
        params = {
            "latitude": selected_station["lat"],
            "longitude": selected_station["lon"],
            "start_date": start_date.strftime("%Y-%m-%d"),
            "end_date": start_date.strftime("%Y-%m-%d"),  # Use the same start and end date for daily data
            "daily": "wind_speed_10m_max",
            "timezone": "Europe/Berlin",
        }

        # Make the API call
        responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)

        for response in responses:
            # Process daily data. The order of variables needs to be the same as requested.
            daily = response.Daily()
            daily_wind_speed_10m_max = daily.Variables(0).ValuesAsNumpy()

            # Calculate the average wind speed for the day
            max_wind_speed = daily_wind_speed_10m_max.max()

            # Append the average wind speed for the day to daily_data
            daily_data.append({
                "date": start_date.strftime('%Y-%m-%d'),  # Format date to include year, month, and day
                "wind_speed_10m_max": max_wind_speed,  # Use the average wind speed
            })

        # Move to the next day
        start_date += pd.Timedelta(days=1)

    # Create a DataFrame from daily data
    daily_dataframe = pd.DataFrame(daily_data)
    
    # Calculate daily averages
    daily_max = daily_dataframe.groupby('date').max()

    return daily_max


In [58]:
def call_open_meteo_temp(selected_station):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)
    
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": selected_station["lat"],
        "longitude": selected_station["lon"],
        "start_date": "1999-12-01",
        "end_date": "1999-12-31",
        "hourly": "temperature_2m",
        "daily": "temperature_2m_mean"
    }
    responses = openmeteo.weather_api(url, params=params)
    
    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    
    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    
    hourly_data = {"date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left",
        normalize=True  # Setze normalize=True, um nur das Datum zu erhalten
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    
    hourly_dataframe = pd.DataFrame(data=hourly_data)
    
    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
    
    daily_data = {"date": pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=False),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=False),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left",
        normalize=True  # Setze normalize=True, um nur das Datum zu erhalten
    )}
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    
    daily_dataframe = pd.DataFrame(data=daily_data)
    
    # Set the index of the DataFrame to the date only
    daily_dataframe.set_index("date", inplace=True)
    
    return daily_dataframe

In [56]:
Deutschland_Berlin={'lat': 52.5200, 'lon': 13.4050}
Deutschland_Berlin_wmean = call_open_meteo_averages(Deutschland_Berlin)
Deutschland_Berlin_wmean

,wind_speed_10m
1999-12-01,28.948442
1999-12-02,27.715563
1999-12-03,28.794542
1999-12-04,25.327858
1999-12-05,20.172232
1999-12-06,22.009834
1999-12-07,25.960112
1999-12-08,22.304487
1999-12-09,20.047384
1999-12-10,18.848070


In [57]:
Deutschland_Berlin={'lat': 52.5200, 'lon': 13.4050}
Deutschland_Berlin_wmin = call_open_meteo_min(Deutschland_Berlin)
Deutschland_Berlin_wmin

,wind_speed_10m_max
date,
1999-12-01,26.341572
1999-12-02,25.241108
1999-12-03,21.749483
1999-12-04,14.843180
1999-12-05,15.307410
1999-12-06,11.592895
1999-12-07,22.218153
1999-12-08,15.913465
1999-12-09,18.504139


In [49]:
Deutschland_Berlin={'lat': 52.5200, 'lon': 13.4050}
Deutschland_Berlin_wmax = call_open_meteo_max(Deutschland_Berlin)
Deutschland_Berlin_wmax

,wind_speed_10m_max
date,
1999-12-01,31.624725
1999-12-02,32.491867
1999-12-03,38.672779
1999-12-04,39.477081
1999-12-05,24.944130
1999-12-06,30.599998
1999-12-07,28.772987
1999-12-08,30.821548
1999-12-09,21.749483


In [59]:
Deutschland_Berlin={'lat': 52.5200, 'lon': 13.4050}
Deutschland_Berlin_tmean = call_open_meteo_temp(Deutschland_Berlin)
Deutschland_Berlin_tmean

,temperature_2m_mean
date,
1999-12-01,7.922582
1999-12-02,4.864250
1999-12-03,5.560083
1999-12-04,3.520500
1999-12-05,1.933000
1999-12-06,1.768417
1999-12-07,7.132999
1999-12-08,5.537167
1999-12-09,7.726749
